In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Accuracy

In [ ]:
# Enable/Disable Eager Execution
# Reference: https://www.tensorflow.org/guide/eager
# TensorFlow's eager execution is an imperative programming environment that evaluates operations immediately, 
# without building graphs

#tf.compat.v1.disable_eager_execution()
#tf.compat.v1.enable_eager_execution()

print("tensorflow version", tf.__version__)
print("keras version", tf.keras.__version__)
print("Eager Execution Enabled:", tf.executing_eagerly())

# Get the number of replicas 
strategy = tf.distribute.MirroredStrategy()
print("Number of replicas:", strategy.num_replicas_in_sync)

devices = tf.config.experimental.get_visible_devices()
print("Devices:", devices)
print(tf.config.experimental.list_logical_devices('GPU'))

print("GPU Available: ", tf.config.list_physical_devices('GPU'))
print("All Physical Devices", tf.config.list_physical_devices())

# Better performance with the tf.data API
# Reference: https://www.tensorflow.org/guide/data_performance
AUTOTUNE = tf.data.experimental.AUTOTUNE

tensorflow version 2.8.0
keras version 2.8.0
Eager Execution Enabled: True
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of replicas: 1
Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]
GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
All Physical Devices [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
hnm_file_format = '/content/drive/MyDrive/Temporary/CS25_Project/HnM_{}.parquet'
hnm_img_format = '/content/drive/MyDrive/Temporary/CS25_Project/images/HnM/{}'

In [ ]:
# folder names
hnm_img_final = '/content/drive/MyDrive/Temporary/CS25_Project/images_final/HnM/'
hnm_img_final_top = '/content/drive/MyDrive/Temporary/CS25_Project/images_final'

In [ ]:
model_wts_path = '/content/drive/MyDrive/Temporary/CS25_Project/'
model_hist_path = '/content/drive/MyDrive/Temporary/CS25_Project/'

In [ ]:
# get list of images
# imgs = [k for k in os.listdir(hnm_img_final)]

In [ ]:
# print(f"We have a total of {len(imgs):,} images")

We have a total of 14,456 images


Getting base model and training the model with our df

In [ ]:
## load the data frame
df = pd.read_parquet(hnm_file_format.format('Final_Input_Df'))
df['Category_Name'] = df['Category_Name'].astype('category')
df['Category_Name_Code'] = df['Category_Name'].cat.codes

In [ ]:
df[['Category_Name', 'Category_Name_Code']].drop_duplicates().sort_values(by='Category_Name_Code')

,Category_Name,Category_Name_Code
16681,Baby_Accessoires,0
15042,Baby_Kleidung,1
22239,Kids_Accessoires,2
16885,Kids_Kleidung,3
21916,Kids_Outdoor,4
22548,Kids_Schuhe,5
14065,Men_Accessoires,6
12335,Men_Cardigans-&-Pullover,7
12481,Men_Hemden,8
12120,Men_Hoodies-&-Sweatshirts,9


In [ ]:
# sort by the code since thats how flow from directory would work
df = df.sort_values(by='Code')
df.head()

,Code,Name,Img_Ext,Parent_Category,Top_Category,Parent_Category_Name,Category_Name,Category_Name_Code
9842,0107727001,Bademantel aus Leinen,.jpg,18.0,Women,Nachtwäsche,Women_Nachtwäsche,23
9844,0107727002,Bademantel aus Leinen,.jpg,18.0,Women,Nachtwäsche,Women_Nachtwäsche,23
9839,0107727021,Bademantel aus Leinen,.jpg,18.0,Women,Nachtwäsche,Women_Nachtwäsche,23
9832,0107727061,Bademantel aus Leinen,.jpg,18.0,Women,Nachtwäsche,Women_Nachtwäsche,23
9837,0107727062,Bademantel aus Leinen,.jpg,18.0,Women,Nachtwäsche,Women_Nachtwäsche,23


In [ ]:
y_train = df['Category_Name_Code'].values
nr_labels = len(np.unique(y_train))

In [ ]:
input_shape = (256, 256, 3)

In [ ]:
random_seed = 109

images_train = image_dataset_from_directory(
    hnm_img_final_top,
    labels = list(y_train),
    subset = 'training',
    validation_split = 0.6,
    seed = random_seed,
    batch_size=256,
    image_size=input_shape[:2]
)

images_val = image_dataset_from_directory(
    hnm_img_final_top,
    labels = list(y_train),
    subset = 'validation',
    validation_split = 0.6,
    seed = random_seed,
    batch_size=256,
    image_size=input_shape[:2]
)


In [ ]:
models = {
    'Resnet50': {
        'model': 'ResNet50',
        'preprocess': 'resnet50',
    },
    'VGG19': {
        'model': 'VGG19',
        'preprocess': 'vgg19',
    },
    'InceptionV3': {
        'model': 'InceptionV3',
        'preprocess': 'inception_v3',
    },
    'MobileNetV2': {
        'model': 'MobileNetV2',
        'preprocess': 'mobilenet_v2',
    },
    'EfficientNetB3': {
        'model': 'EfficientNetB3',
        'preprocess': 'efficientnet',
    },
}

In [ ]:
# add a global spatial average pooling layer

epochs = 50
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

model_hists = {}

for model_name, model_params in models.items():
  if os.path.exists(f"{model_hist_path}{model_name}.pkl"):
    # we've completed this one... let's keep going
    print(f"Skipping {model_name} since this is already completed.")
    continue

  print(f"Starting training with {model_name}")
  inputs = tf.keras.layers.Input(shape=input_shape)
  # We make sure that the base_model is running in inference mode here,
  # by passing `training=False`. This is important for fine-tuning, as you will
  # learn in a few paragraphs.
  base_model = eval('tf.keras.applications.' + model_params['model'])(include_top=False, input_shape=input_shape, pooling='avg')
  # x = base_model(inputs, training=False)
  x = base_model.output
  # x = base_model.get_layer('avg_pool').output
  # x = GlobalAveragePooling2D()(x)
  # let's add a fully-connected layer
  x = Dense(256, activation='relu')(x)
  x = Dropout(0.5)(x)
  x = Dense(256, activation='relu')(x)
  # and a logistic layer -- with classes = nr_labels
  predictions = Dense(nr_labels, activation='softmax')(x)

  # this is the model we will train
  model = Model(inputs=base_model.input, outputs=predictions)

  # first: train only the top layers (which were randomly initialized)
  # i.e. freeze all convolutional InceptionV3 layers
  for layer in base_model.layers:
      layer.trainable = False

  # compile the model (should be done *after* setting layers to non-trainable)
  model.compile(optimizer=Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  # model.compile(optimizer=Adam(learning_rate=1e-3), loss=SparseCategoricalCrossentropy(), metrics=[Accuracy])

  # preprocess train and test sets if required
  if model_params.get('preprocess', False):
    images_train_f = images_train.map(lambda x, y: ( eval('tf.keras.applications.' + model_params['preprocess'] + '.preprocess_input')(x), y ), num_parallel_calls=AUTOTUNE)
    images_val_f = images_val.map(lambda x, y: ( eval('tf.keras.applications.' + model_params['preprocess'] + '.preprocess_input')(x), y ), num_parallel_calls=AUTOTUNE)
  else:
    images_train_f = images_train
    images_val_f = images_val

  images_train_f = images_train_f.prefetch(1)
  images_val_f = images_val_f.prefetch(1)

  print(f"All ready! Starting fit!")
  # model.summary()
  hist = model.fit(
      images_train_f,
      batch_size=256,
      epochs=epochs,
      validation_data=images_val_f,
      verbose=True,
      callbacks=[es]
    )
  
  model.save_weights(f"{model_wts_path}{model_name}.h5")
  
  min_loss_idx = np.argmin(hist.history['val_loss'])

  print(f"Completed training of {model_name} with max validation accuracy: {hist.history['val_accuracy'][min_loss_idx]:.2%} at Epoch Nr {min_loss_idx+1}")

  model_hists[model_name] = hist
  with open(f"{model_hist_path}{model_name}.pkl", 'wb') as file_pi:
      pickle.dump(hist.history, file_pi)
  

Skipping Resnet50 since this is already completed.
Skipping VGG19 since this is already completed.
Starting training with InceptionV3
87924736/87910968 [==============================] - 0s 0us/step
All ready! Starting fit!
Epoch 1/50
23/23 [==============================] - 236s 2s/step - loss: 3.0222 - accuracy: 0.2488 - val_loss: 2.6651 - val_accuracy: 0.3022
Epoch 2/50
23/23 [==============================] - 50s 2s/step - loss: 2.6222 - accuracy: 0.3211 - val_loss: 2.3702 - val_accuracy: 0.3615
Epoch 3/50
23/23 [==============================] - 50s 2s/step - loss: 2.3519 - accuracy: 0.3714 - val_loss: 2.1262 - val_accuracy: 0.4150
Epoch 4/50
23/23 [==============================] - 50s 2s/step - loss: 2.1431 - accuracy: 0.4067 - val_loss: 1.9236 - val_accuracy: 0.4566
Epoch 5/50
23/23 [==============================] - 50s 2s/step - loss: 1.9632 - accuracy: 0.4363 - val_loss: 1.7769 - val_accuracy: 0.4892
Epoch 6/50
23/23 [==============================] - 50s 2s/step - loss: 1.8